In [1]:
import pandas as pd
import numpy as np
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
import ast
import warnings
import regex as re

warnings.filterwarnings('ignore')
from decimal import *

from enrich_dimensions.rounds import rounds, rounding, re_extract, curate, round_string_float     
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

In [2]:
customer_id = '137'
customer_name='%kimballwest%'
buckets = "Fixings & Fasteners"
# strategy_version_input=705
# attribute_id_input=5131
formatted_attribute = 'diameter_size'

In [3]:
# parameters
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [4]:
import json
dfs = query_from_file(file_name='./query/custom_fields.sql', params=params)
custom_field_df=pd.json_normalize(dfs['custom_fields'])
fields = ['Diameter']
df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df[fields]], axis = 1)
# df = pd.concat([dfs.drop('custom_fields', axis=1)], axis = 1)
print(len(df))

del df['url']
del df['image_url']
del df['value']
del df['product_id']
del df['bucket_id']

5201


In [5]:
import collections
print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])

[]


In [16]:
pat_frac=rf'''(?i)(\#\d+.?\d*\/?\d*)|(\#\d+.?\#\d+)|(M\d+\.?\d*\s?.?\s?M?\d*\.?\d*)|(T\d+\.?\d*.{0,3}T?\d*\.?\d*)|()'''         
df['name'] = df['product_name'].apply(lambda x: re_extract(pat_frac, str(x)))
df['ld'] = df['long_desc'].apply(lambda x: re_extract(pat_frac, str(x)))


pat=r'(?i)(\#\d+.?\d*\/?\d*)|(\#\d+.?\#\d+)|(M\d+\.?\d*\s?.?\s?M?\d*\.?\d*)|(T\d+\.?\d*.{0,3}T?\d*\.?\d*)|()'
df['matches']=df['Diameter'].apply(lambda x: re_extract(pat, str(x)))
custom=df[df['matches'].astype(str)!='[]']
custom[curation_col]=custom['matches'].apply(lambda x: re.sub(r'.+to\s?|"','',str(x))).apply(lambda x: re.sub(r"'",'"',str(x)))
matchcustom=custom[['external_id',curation_col]]

In [17]:
two=df[df['matches'].astype(str)=='[]']
pat_diam=rf'''(?i)(.{0,10}\#\d+.?\d*\/?\d*.{0,10})|(\#\d+.?\#\d+)|(M\d+\.?\d*\s?.?\s?M?\d*\.?\d*)|(T\d+\.?\d*.{0,3}T?\d*\.?\d*)|()''' 
two['matches'] = two['long_desc'].apply(lambda x: re_extract(pat_diam, str(x)))
two['matches'].explode().value_counts()

M4-003.         6
M12 & 1         1
M14 x 20        1
M12-1           1
M16.4 N         1
M14F            1
M6.3,           1
M16.8 x 26.8    1
M14 m           1
Name: matches, dtype: int64

# N/A

In [18]:
na=two[two['matches'].astype(str)=='[]']
na[curation_col]='n/a'
matchna=na[['external_id',curation_col]]
print(len(na)+len(matchcustom))

5187


In [24]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchcustom))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchna))


# def get_df_name(df):
#     name =[x for x in gglobals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchfinal))

# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchex))

matchcustom
matchna


In [12]:
stop

NameError: name 'stop' is not defined

# send to the folder for upload

In [25]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchcustom) 

In [26]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchna) 